# Starting data analysis

In [9]:
import pandas as pd
import re

In [10]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def refactorTime(name, df):
    df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: x + ":00" if (re.search("^((\d\d)/(\d\d)/(\d\d\d\d)|(\d\d\d\d)-(\d\d)-(\d\d)) (\d\d):([0-5][0-9]):([0-5][0-9])$", x) == None) else x)
    try:
        df.iloc[:, 1] = pd.to_datetime(df.iloc[:, 1], format="%m/%d/%Y %H:%M:%S")
    except:
        df.iloc[:, 1] = pd.to_datetime(df.iloc[:, 1], format="%Y-%m-%d %H:%M:%S")
    # print(name)
    df.iloc[:, 2] = df.iloc[:, 2].apply(pd.to_numeric, errors='coerce')

    # set index to time, this makes df a time series df and then apply pandas time series functions.  
    df.set_index(df.iloc[:, 1], drop=True, inplace=True)

    # create another df by resampling the original df
    new_df = pd.DataFrame(df.iloc[:, 2].resample("5min", label="right", closed="right").mean().pad())
    new_df = new_df.rename(columns={" Reading": name + "Reading", " Units": name + "Reading", "value": "stepCount", " Carb Content": "carbCount"})
    new_df = new_df.rename_axis("time")
    return new_df

# # DEPRECATED
# basal_52104 = pd.read_csv("Data/PatientData/EAOB/52104/basal.csv")
# bolus_52104 = pd.read_csv("Data/PatientData/EAOB/52104/bolus.csv")
# cgm_52104 = pd.read_csv("Data/PatientData/EAOB/52104/cgm.csv")
# hr_52104 = pd.read_csv("Data/PatientData/EAOB/52104/hr.csv")
# meal_52104 = pd.read_csv("Data/PatientData/EAOB/52104/meal.csv")
# smbg_52104 = pd.read_csv("Data/PatientData/EAOB/52104/smbg.csv")
# smbgcal_52104 = pd.read_csv("Data/PatientData/EAOB/52104/smbgcal.csv")
# steps_52104 = pd.read_csv("Data/PatientData/EAOB/52104/steps.csv")
# dataFrames = [basal_52104, bolus_52104, cgm_52104, smbg_52104, smbgcal_52104, steps_52104, meal_52104]

# dataFrames_1 = []
# for frame in dataFrames:
#     frame = refactorTime(frame)
#     # print(frame.head())
#     dataFrames_1.append(frame)
# df_52104 = pd.concat(dataFrames_1, axis=1)
# # print(df_52104.head())
# df_52104.to_csv("df_52104.csv")

In [11]:
basal = pd.DataFrame()
bolus = pd.DataFrame()
cgm = pd.DataFrame()
meal = pd.DataFrame()
smbg = pd.DataFrame()
smbgcal = pd.DataFrame()
steps = pd.DataFrame()
d = {"basal": basal, "bolus": bolus, "cgm": cgm, "meal": meal, "smbg": smbg, "smbgcal": smbgcal, "steps": steps}
def configurePatient(patientNum):
    dataFrames_1 = []
    for key, value in d.items():
        value = pd.read_csv('Data/PatientData/EAOB/' + str(patientNum) + '/' + key + '.csv')
        value = refactorTime(key, value)
        dataFrames_1.append(value)
    df_patient = pd.concat(dataFrames_1, axis=1)
    return df_patient

In [12]:
patientList = range(52104, 52122)
df_52104 = pd.DataFrame()
df_52106 = pd.DataFrame()
df_52107 = pd.DataFrame()
df_52108 = pd.DataFrame()
df_52109 = pd.DataFrame()
df_52110 = pd.DataFrame()
df_52111 = pd.DataFrame()
df_52113 = pd.DataFrame()
df_52116 = pd.DataFrame()
df_52117 = pd.DataFrame()
df_52118 = pd.DataFrame()
df_52119 = pd.DataFrame()
df_52121 = pd.DataFrame()
df_52122 = pd.DataFrame()


patientDictionary = {52104: df_52104, 52106: df_52106, 52107: df_52107, 52108: df_52108, 52109: df_52109, 52110: df_52110, 52111: df_52111, 52113: df_52113, 52116: df_52116, 52117: df_52117, 52118: df_52118, 52119: df_52119, 52121: df_52121, 52122: df_52122}
dataFrames_2 = []
for key, value in patientDictionary.items():
    # print(key)
    value = configurePatient(key)
    dataFrames_2.append(value)
    # print(value.head())
dataFrames_2[0].to_csv("df_52104.csv")
dataFrames_2[1].to_csv("df_52106.csv")
dataFrames_2[2].to_csv("df_52107.csv")
dataFrames_2[3].to_csv("df_52108.csv")
dataFrames_2[4].to_csv("df_52109.csv")
dataFrames_2[5].to_csv("df_52110.csv")
dataFrames_2[6].to_csv("df_52111.csv")
dataFrames_2[7].to_csv("df_52113.csv")
dataFrames_2[8].to_csv("df_52116.csv")
dataFrames_2[9].to_csv("df_52117.csv")
dataFrames_2[10].to_csv("df_52118.csv")
dataFrames_2[11].to_csv("df_52119.csv")
dataFrames_2[12].to_csv("df_52121.csv")
dataFrames_2[13].to_csv("df_52122.csv")
